In [ ]:
!pip install pandas
!pip install pydub

In [ ]:
import glob
import os
import pandas as pd
from pydub import AudioSegment

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
insighLab_folder = '/content/drive/MyDrive/sss/voxforgeDS'

## Voxforge

In [ ]:
# Appends the folder to the name of the file to avoid duplicate names
def rename_files(treeroot):
    for dir,_,files in os.walk(treeroot):
        for f in files: 
            ext = os.path.splitext(f)[1]
            if ext == '.wav':
                old_name = os.path.realpath( os.path.join(dir,f) )
                append_name = dir.split("/")[-2]                
                new_name = os.path.join(dir, f'{append_name}_{f}')
                os.rename(old_name, new_name)


def create_tsv(treeroot):
    save_to = '/content/drive/MyDrive/sss/trySentences.tsv'
    # Clear previous file
    output_file = open(save_to, "w")
    output_file.write(f'File\tsentence\tgender\n')
    output_file.close()

    print(treeroot)
    
    for dirpath, dirnames, filenames in os.walk(treeroot):
        print(f'dirpath:{dirpath}\ndirnames:{dirnames}\nfilenames:{filenames}\n')
        infos = pd.DataFrame(columns=['file', 'sentence', 'gender'])
        if dirpath.split("/")[-1] == 'wav':
            infos.file = filenames
            parent_folder = dirpath.split("/wav")[0]
            infos.gender = get_gender(f'{parent_folder}/etc/README')
            # print(f'dirpath:{dirpath}\ndirnames:{dirnames}\nfilenames:{filenames}\n')

            with open(f'{parent_folder}/etc/prompts-original') as file:
                for line in file.readlines():
                    line = line.rstrip('\n')
                    ground_truth = line.split(" ", 1)
                    unique_folder_name = dirpath.split("/")[-2] 
                    wav_name = f'{unique_folder_name}_{ground_truth[0]}.wav'
                    infos.loc[infos['file'] == wav_name, 'sentence'] = ground_truth[1]
            
            # Save current infos
            output_file = open(save_to, "a")
            for info in infos.values:
                output_file.write(f'{info[0]}\t{info[1]}\t{info[2]}\n')
            output_file.close()
            

def get_gender(pathfile):
    """Os gêneros existentes são:
        Male, Masculino, Feminino, desconhecido, Selecione e Vazio

    Args:
        pathfile ([type]): [description]

    Returns:
        [type]: [description]
    """
    token = "Gender: "
    with open(pathfile) as file:
        for line in file.readlines():
            line = line.rstrip('\n')
            if token in line:
                return line.split(token)[-1]
        return "Vazio"

In [ ]:
%cd {insighLab_folder}
!pwd

/content/drive/.shortcut-targets-by-id/15zk-dG1j3hGlXSI-e8wchudigw8Fgibu/sss/voxforgeDS
/content/drive/.shortcut-targets-by-id/15zk-dG1j3hGlXSI-e8wchudigw8Fgibu/sss/voxforgeDS


In [ ]:

%%bash
!/bin/bash

wget -v -r -l1  http://www.repository.voxforge1.org/downloads/pt/Trunk/Audio/Original/48kHz_16bit/
mkdir -p voxforge/wav

In [ ]:
%%bash
!/bin/bash

cd www.repository.voxforge1.org/
# Remove samples in English
find . -name 'anonymous-20131016-uzv*' -exec rm '{}' \;
find . -name 'anonymous-20140619-wcy*' -exec rm '{}' \;

# Descompacta
find . -name '*.tgz' -exec tar zxvf '{}' \;

In [ ]:
res = os.walk("/content/drive/MyDrive/sss/www.repository.voxforge1.org/")

In [ ]:
# Cria nomes únicos
# rename_files("/content/drive/MyDrive/sss/www.repository.voxforge1.org/")
# Cria arquivo de sentenças
create_tsv("/content/drive/MyDrive/sss/www.repository.voxforge1.org/")

/content/drive/MyDrive/sss/www.repository.voxforge1.org/


In [ ]:
!pwd

/content/drive/.shortcut-targets-by-id/15zk-dG1j3hGlXSI-e8wchudigw8Fgibu/sss/voxforgeDS


In [ ]:
%%bash

cd www.repository.voxforge1.org/

# Move arquivos wav de interesse para pasta apropriada
find . -name '*.wav' -exec cp --backup=t '{}' ../voxforge/wav \;

Process is interrupted.


In [ ]:
!zip -r voxforge.zip voxforge
# !cp voxforge.zip '/content/drive/My Drive/TCC_data/corpus/'
!cp voxforge.zip {insighLab_folder}"/datasets/"